# Clean data from video annotator

Create a script to clean data from video annotator

install python video annotator: http://pythonvideoannotator.readthedocs.io/en/master/

## Run Video Annotator in Terminal

### for windows:
- activate videoannotator
- python -m pythonvideoannotator

### for mac:
- source activate videoannotator
- python -m pythonvideoannotator

## List of tasks
self.taskList = np.array(['Shaking', 'Standing', 'Walking', 'Walking while counting', 'Finger to nose--right hand', 
                          'Finger to nose--left hand', 'Alternating right hand movements', 
                          'Alternating left hand movements', 'Sit to stand', 'Drawing on a paper', 
                          'Typing on a computer keyboard', 'Assembling nuts and bolts', 'Taking a glass of water',
                          'Organizing sheets in a folder', 'Folding towels', 'Sitting'])

Shaking
Standing
Walking
Walking while counting
Finger to nose--right hand
Finger to nose--left hand
Alternating right hand movements
Alternating left hand movements
Sit to stand
Drawing on a paper
Typing on a computer keyboard
Assembling nuts and bolts
Taking a glass of water
Organizing sheets in a folder
Folding towels
Sitting

In [ ]:
# For Joe's computer
# cd Y:

In [1]:
# Importing the Libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re

#import datetime as dt
#import pathlib
#import pickle
#from itertools import product
#from scipy.stats import skew, kurtosis, pearsonr
#from scipy.signal import butter, welch, filtfilt, resample
#import time
#import copy

In [2]:
# specify path
#path = r'C:\Users\lu-rt&o\Documents\GitHub\CIS-PD MUSC\CIS-PD-MUSC'
#path = r'C:\Users\lu-rt&o\pythonVideoAnnotator'
path = r'Y:\CIS-PD Videos\Cisnwh8'

## Videoannotator rows correspond to video cycle number

In [ ]:
# add subject id column

In [3]:
def dataclean(file):    
    colnames = ['subject code', '1', 'start time', 'stop time', 'activity', '2', 'cycle']
    selectedcol= ['subject code','start time', 'stop time', 'activity', 'cycle']
    #df = pd.read_csv(os.path.join(path, 'cycle1.csv'), names=colnames, usecols=selectedcol)
    filename = os.path.join(path, file) # ie. file = 'cycle1.csv'
    df = pd.read_csv(filename, names=colnames, usecols=selectedcol)
    df = df.dropna()
    df['cycle'] += 1
    df['stop time'] = pd.to_numeric(df['stop time'], downcast='integer')
    df['cycle'] = pd.to_numeric(df['cycle'], downcast='integer')
    df['subject code'] = 1030
    print(df.head(3))
    
    return df

In [22]:
df1 = dataclean('cycle1.csv')
df2 = dataclean('cycle2.csv')
df3 = dataclean('cycle3incomplete.csv')
df4 = dataclean('cycle36.csv')

   subject code start time  stop time                activity  cycle
1          1030       1118       2145                Standing      1
2          1030       2315       3333                 Walking      1
3          1030       3608       4800  Walking while counting      1
   subject code start time  stop time                activity  cycle
2          1030       1088       2037                Standing      2
3          1030       2251       3262                 Walking      2
4          1030       3307       4423  Walking while counting      2
   subject code start time  stop time                activity  cycle
3          1030       1228       2230                Standing      3
4          1030       2381       3572                 Walking      3
5          1030       3572       4630  Walking while counting      3
   subject code start time  stop time                       activity  cycle
3          1030      12623      13501        Taking a glass of water      3
4          1030     

In [23]:
frames = [df1, df2, df3, df4]
df = pd.concat(frames, ignore_index=True)

In [24]:
print(df.head(5))
print(df.tail(5))

   subject code start time  stop time                    activity  cycle
0          1030       1118       2145                    Standing      1
1          1030       2315       3333                     Walking      1
2          1030       3608       4800      Walking while counting      1
3          1030       5832       6518  Finger to nose--right hand      1
4          1030       6518       7121   Finger to nose--left hand      1
    subject code start time  stop time                       activity  cycle
85          1030      11348      12775      Assembling nuts and bolts      6
86          1030      13017      13871        Taking a glass of water      6
87          1030      14646      16008  Organizing sheets in a folder      6
88          1030      16537      17627                 Folding towels      6
89          1030      17718      18765                        Sitting      6


In [31]:
filename2 = os.path.join(path, '1030annotated.csv')
with open(filename2,'wb') as f:
    df.to_csv(filename2, sep=',')